<a href="https://colab.research.google.com/github/pluieciel/applied-ml-uni-lu/blob/master/Copy_of_automatic1111_stable_diffusion_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Stable Diffusion WebUI Colab](https://github.com/ddPn08/stable-diffusion-webui-colab) by [ddPn08](https://github.com/ddpn08/)

This colab runs from the repo [`automatic1111`](https://github.com/AUTOMATIC1111/stable-diffusion-webui)  
このColabは[`automatic1111`](https://github.com/AUTOMATIC1111/stable-diffusion-webui) を使用しています。


In [ ]:
! nvidia-smi
! nvcc -V
! free -h

Sun Dec  4 15:25:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1 - Setup

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### 1.1 Clone repository

In [ ]:
%cd /content/drive/MyDrive/AI
repository_url = "https://github.com/AUTOMATIC1111/stable-diffusion-webui"  # @param {type: "string"}
webui_branch = "master"  # @param {type: "string"}

! git clone {repository_url}
%cd /content/drive/MyDrive/AI/stable-diffusion-webui
! git checkout {webui_branch}

/content/drive/MyDrive/AI
fatal: destination path 'stable-diffusion-webui' already exists and is not an empty directory.
/content/drive/MyDrive/AI/stable-diffusion-webui
D	embeddings/Place Textual Inversion embeddings here.txt
D	models/VAE/Put VAE here.txt
M	webui.sh
Already on 'master'
Your branch is up to date with 'origin/master'.


### 1.2 Setup models

In [ ]:
# @markdown # Load the stable-diffusion model
# @markdown > ### stable-diffusionのモデルをロード

# @markdown **Model Path Variables**
# ask for the link
%cd /content/drive/MyDrive/AI
print("Local Path Variables:\n")

model_filename = "/content/drive/MyDrive/AI"  # @param {type:"string"}
models_path = "/content/drive/MyDrive/AI"  # @param {type:"string"}
output_path = "/content/drive/MyDrive/AI/outputs"  # @param {type:"string"}
config_path = "/content/drive/MyDrive/AI/automatic1111/config"  # @param {type:"string"}
extensions_file_path = ""

# @markdown **Optional | Download the model if it isn't already in the `models_path` folder**
# @markdown > モデルが`models_path`フォルダーにない場合に、モデルをダウンロードする

# @markdown To download the model, you need to have accepted the terms [HERE](https://huggingface.co/CompVis/stable-diffusion-v-1-4-original)
# @markdown and have copied a token from [HERE](https://huggingface.co/settings/tokens)

# @markdown > モデルをダウンロードするには, [このページ](https://huggingface.co/CompVis/stable-diffusion-v-1-4-original)で条件に同意する必要があります。
# @markdown > また、[このページ](https://huggingface.co/settings/tokens)からトークンを取得する必要があります。
download_if_missing = False  # @param {type:"boolean"}
model_url = ""  # @param {type:"string"}
token = ""  # @param {type:"string"}

# @markdown **Optional | Google Drive Path Variables**
mount_google_drive = True  # @param {type:"boolean"}
force_remount = False

import os

mount_success = True
if mount_google_drive:
    from google.colab import drive

    try:
        drive_path = "/content/drive"
        drive.mount(drive_path, force_remount=force_remount)
        models_path_gdrive = "/content/drive/MyDrive/AI"  # @param {type:"string"}
        output_path_gdrive = "/content/drive/MyDrive/AI/outputs"  # @param {type:"string"}
        config_path_gdrive = "/content/drive/MyDrive/AI/automatic1111/config"  # @param {type:"string"}
        extensions_file_path_gdrive = "/content/drive/MyDrive/AI/automatic1111/extensions.txt"  # @param {type:"string"}

        models_path = models_path_gdrive
        output_path = output_path_gdrive
        config_path = config_path_gdrive
        extensions_file_path = extensions_file_path_gdrive
    except:
        print("...error mounting drive or with drive path variables")
        print("...reverting to default path variables")
        mount_success = False

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(config_path, exist_ok=True)
os.makedirs(f"{models_path}/embeddings", exist_ok=True)
os.makedirs(f"{models_path}/VAE", exist_ok=True)
os.makedirs(f"{models_path}/hypernetworks", exist_ok=True)

! rm -Rf stable-diffusion-webui/embeddings && ln -s {models_path}/embeddings stable-diffusion-webui/embeddings
! rm -Rf stable-diffusion-webui/models/VAE && ln -s {models_path}/VAE stable-diffusion-webui/models/VAE
! rm -Rf stable-diffusion-webui/models/hypernetworks && ln -s {models_path}/hypernetworks stable-diffusion-webui/models/hypernetworks

if os.path.exists("stable-diffusion-webui/extensions/stable-diffusion-webui-aesthetic-gradients"):
    os.makedirs(f"{models_path}/aesthetic_embeddings", exist_ok=True)
    ! rm -Rf stable-diffusion-webui/extensions/stable-diffusion-webui-aesthetic-gradients/aesthetic_embeddings \
        && ln -sf {models_path}/aesthetic_embeddings stable-diffusion-webui/extensions/stable-diffusion-webui-aesthetic-gradients/aesthetic_embeddings

if download_if_missing:
    if not mount_success:
        print("Downloading model to " + models_path + " due to gdrive mount error")
    elif not os.path.exists(models_path + "/" + model_filename):
        ! mkdir sd-model
        %cd /content/sd-model/
        ! curl -LJ  {model_url} -o {model_filename} {'-H "Authorization: Bearer ' + token + '"' if token else ""}
        ! mv /content/sd-model/{model_filename} {models_path}/
        %cd /content/
    else:
        print("Model already downloaded, moving to next step")

%cd /content/drive/MyDrive/AI/stable-diffusion-webui
! curl -LOJ https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")
print(f"config_path: {config_path}")

/content/drive/MyDrive/AI
Local Path Variables:

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/AI/stable-diffusion-webui
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Refusing to overwrite GFPGANv1.3.pth: File exists
  0  332M    0   873    0     0   1408      0 68:46:48 --:--:-- 68:46:48 11192
curl: (23) Failed writing body (0 != 873)
models_path: /content/drive/MyDrive/AI
output_path: /content/drive/MyDrive/AI/outputs
config_path: /content/drive/MyDrive/AI/automatic1111/config


## 2 Optional - Set webUI settings and configs before running

In [ ]:
# @markdown # Advanced | Launch preferences
# @markdown # 詳細設定

# @markdown &nbsp;

# @markdown ## Extensions
load_extensions_from_gdrive = False  # @param {type:"boolean"}
extensions = "https://github.com/yfszzx/stable-diffusion-webui-images-browser, https://github.com/DominikDoom/a1111-sd-webui-tagcomplete"  # @param {type:"string"}
extensions = list(map(str.strip, extensions.split(',')))

if load_extensions_from_gdrive and extensions_file_path:
    if os.path.exists(extensions_file_path):
        with open(extensions_file_path, mode="r") as f:
            for s in f:
                url = s.strip()
                if url not in extensions:
                    extensions.append(url)
    with open(extensions_file_path, mode="w+") as f:
        f.write("\n".join(extensions))

# @markdown &nbsp;
# @markdown ## Command line arguments


no_half = False  # @param {type:"boolean"}
no_half_vae = False # @param {type:"boolean"}
allow_code = False # @param {type:"boolean"}
no_progressbar_hiding = False  # @param {type:"boolean"}
medvram = False  # @param {type:"boolean"}
lowvram = False  # @param {type:"boolean"}
deepdanbooru = False # @param {type:"boolean"}
xformers = True  # @param {type:"boolean"}
disable_opt_split_attention = False  # @param {type:"boolean"}

# @markdown &nbsp;
custom_arguments = "--enable-console-prompts" # @param {type:"string"}

run_string_with_variables = {
    "--no-half": f"{no_half}",
    "--no-half-vae": f"{no_half_vae}",
    "--allow-code": f"{allow_code}",
    "--no-progressbar-hiding": f"{no_progressbar_hiding}",
    "--medvram": f"{medvram}",
    "--lowvram": f"{lowvram}",
    "--deepdanbooru": f"{deepdanbooru}",
    "--xformers": f"{xformers}",
    "--disable-opt-split-attention": f"{disable_opt_split_attention}",
}

advanced_options = {k for (k, v) in run_string_with_variables.items() if v == "True"}

# @markdown &nbsp;
# @markdown ## Optional | Ngrok Tunnel
# @markdown > オプション | Ngrokのトンネルを起動する

# @markdown Use Ngrok tunneling for more stable communication. To use this, you need to [sign up](https://ngrok.com/) for Ngrok and [get a token](https://dashboard.ngrok.com/get-started/setup).
# @markdown > より安定した通信のために Ngrok トンネリングを使用します。 これを使用するには、Ngrok に[サインアップ](https://ngrok.com/) して[トークンを取得](https://dashboard.ngrok.com/get-started/setup) する必要があります。

use_ngrok = False  # @param {type: "boolean"}
ngrok_auth_token = ""  # @param {type: "string"}
ngrok_region = "us"  # @param ["us", "eu", "au", "ap", "sa", "jp", "in"]

# @markdown &nbsp;
# @markdown ## Enable password authentication (Prevent other users from using the WebUI)
# @markdown > パスワード認証を有効にする (WebUIが他のユーザーによって使用されないようにします)

use_gradio_auth = False # @param {type:"boolean"}
gradio_auth_username = "username" # @param {type:"string"}
gradio_auth_password = "password" # @param {type:"string"}

# @markdown &nbsp;
# @markdown ## First load the model specified by `model_filename`
# @markdown > `model_filename`で指定したモデルをはじめにロードする
specify_model = False # @param {type:"boolean"}

vars = " ".join(advanced_options)
if not use_ngrok:
    vars += " --share"
else:
    vars += f" --ngrok {ngrok_auth_token} --ngrok-region {ngrok_region}"

if specify_model:
    vars += f" --ckpt {models_path}/{model_filename}"

if use_gradio_auth:
    vars += f" --gradio-auth {gradio_auth_username}:{gradio_auth_password}"


import os

os.environ['COMMANDLINE_ARGS'] = f"""
--ckpt-dir {models_path} \
--ui-config-file {config_path}/ui-config.json \
--ui-settings-file {config_path}/config.json \
--styles-file {config_path}/styles.csv \
{vars} \
{custom_arguments}
"""


## 3 - Launch WebUI

In [ ]:
# @markdown ## Setup environment
# @markdown This may take up to 10 minutes
# @markdown > これには10分ほどかかる場合があります。

# @markdown &nbsp;
store_env_gdrive = True # @param{type:"boolean"}
# @markdown Use about 3GB of GoogleDrive capacity and store Conda env on Google Drive. (Setup will be faster next time)
# @markdown > GoogleDriveの容量を約3GB使い、Condaの環境データをGoogleドライブに保存する。 (次回からのセットアップが早くなります)

# @markdown &nbsp;
force_reinstall_environmemt = False # @param{type:"boolean"}
# @markdown Do not use environments stored on GoogleDrive.
# @markdown > GoogleDriveに保存されている環境を使用しない。
gdrive_env_directory = "/content/drive/MyDrive/AI/automatic1111/conda-env" # @param{type:"string"}
gdrive_env_file = f"{gdrive_env_directory}/env.tar.zst"


%cd /content/drive/MyDrive/AI/stable-diffusion-webui/extensions

for extension in extensions:
    ! git clone {extension}
    extension_name = extension.split("/")[-1]
    ! cd {extension_name} && git fetch

%cd /content/drive/MyDrive/AI
import os
import sys

! apt update
! apt upgrade -y
! apt install zstd -y

if not os.path.exists("/usr/local/bin/conda"):
    ! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    ! chmod +x Miniconda3-latest-Linux-x86_64.sh
    ! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
    ! rm Miniconda3-latest-Linux-x86_64.sh

if os.path.exists(gdrive_env_file) and not os.path.exists("/usr/local/envs/automatic") and not force_reinstall_environmemt:
    os.makedirs("/usr/local/envs/automatic", exist_ok=True)
    ! zstd -dc {gdrive_env_file} | tar -xf - -C /usr/local/envs/automatic
    update_script = """#!/bin/bash
    eval "$(conda shell.bash hook)"
    cd stable-diffusion-webui
    conda env update -n automatic -f ./environment-wsl2.yaml
    conda activate automatic
    conda install -y xformers -c xformers/label/dev
    python3 -m pip install --upgrade tensorrt
    """
    ! {update_script}
elif not os.path.exists("/usr/local/envs/automatic"):
    install_script = """#!/bin/bash
    eval "$(conda shell.bash hook)"
    cd stable-diffusion-webui
    conda env create -f ./environment-wsl2.yaml
    conda activate automatic
    conda install -y xformers -c xformers/label/dev
    python3 -m pip install --upgrade tensorrt
    python -c 'from launch import prepare_enviroment; prepare_enviroment()'"""
    ! {install_script}

os.environ["LD_LIBRARY_PATH"] = f"{os.environ['LD_LIBRARY_PATH']}:/usr/local/envs/automatic/lib"

if store_env_gdrive and mount_google_drive and mount_success:
    import threading
    def pack():
        os.makedirs(gdrive_env_directory, exist_ok=True)
        ! tar -C /usr/local/envs/automatic -cf - . | zstd > /content/tmp.tar.zst
        ! mv /content/tmp.tar.zst {gdrive_env_file}
        print("Finish storing environment.")
    threading.Thread(target=pack).start()
    print("Started storing the conda environment in a separate thread. This will take approximately 2 minutes.")

/content/drive/MyDrive/AI/stable-diffusion-webui/extensions
fatal: destination path 'stable-diffusion-webui-images-browser' already exists and is not an empty directory.
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/yfszzx/stable-diffusion-webui-images-browser
   1552855..eb7e042  main       -> origin/main
fatal: destination path 'a1111-sd-webui-tagcomplete' already exists and is not an empty directory.
/content/drive/MyDrive/AI
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning

In [ ]:
# @markdown # Run script
# @markdown keep in mind that this script is set to run for ever.
# @markdown > ※注意 このスクリプトは永久に実行されます。

import os

transformers_offline = False # @param {type: "boolean"}
os.environ['TRANSFORMERS_OFFLINE'] = "1" if transformers_offline else "0"

# @markdown &nbsp;

# @markdown ### Important - click the public URL to launch WebUI in another tab
# @markdown > ### 重要 - 公開URLをクリックしてWebUIを起動してください

# @markdown ![](https://user-images.githubusercontent.com/71378929/189563599-6df78bcf-133b-41e8-a55d-8ca3783cd933.png)

%cd /content/drive/MyDrive/AI/stable-diffusion-webui/
! git pull

run_script = """#!/bin/sh
eval "$(conda shell.bash hook)"
conda activate automatic
python launch.py"""
! {run_script}

/content/drive/MyDrive/AI/stable-diffusion-webui
Updating 4b3c5bc..44c46f0
error: Your local changes to the following files would be overwritten by merge:
	webui.sh
Please commit your changes or stash them before you merge.
Aborting
Python 3.10.8 (main, Nov 24 2022, 14:13:03) [GCC 11.2.0]
Commit hash: 4b3c5bc24bffdf429c463a465763b3077fe55eb8
Installing requirements for Web UI
Launching Web UI with arguments: --ckpt-dir /content/drive/MyDrive/AI --ui-config-file /content/drive/MyDrive/AI/automatic1111/config/ui-config.json --ui-settings-file /content/drive/MyDrive/AI/automatic1111/config/config.json --styles-file /content/drive/MyDrive/AI/automatic1111/config/styles.csv --xformers --share --enable-console-prompts
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
Loading weights [b8dabc19] from /content/drive/MyDrive/AI/sammod_0.75-anythingv3_0.25-Weighted_sum-merged.ckpt
Applying xformers cross attention optimization.
Model loaded.
Loaded a total of 0